In [1]:
from queue import Queue
from threading import Thread
import random

def read_vocab(vocab_file):
    f = open(vocab_file, 'rb')
    vocabs = [line.decode('utf8')[:-1] for line in f]
    f.close()
    return vocabs

vocab_file = 'C:\\Users\\16010\\source\\repos\\seq2seq_1_py\\seq2seq_1_py\\data\\dl-data\\couplet\\vocabs'
input_file ='C:\\Users\\16010\\source\\repos\\seq2seq_1_py\\seq2seq_1_py\\data\\dl-data\\couplet\\train\\in.txt'
target_file = 'C:\\Users\\16010\\source\\repos\\seq2seq_1_py\\seq2seq_1_py\\data\\dl-data\\couplet\\train\\out.txt'


vocabs = read_vocab(vocab_file)

data = []

input_f = open(input_file, 'rb')
target_f = open(target_file, 'rb') 

i = 0
data = []
for input_line in input_f:
    input_line = input_line.decode('utf-8')[:-1]
    target_line = target_f.readline().decode('utf-8')[:-1]
    input_words = [x for x in input_line.split(' ') if x != '']
    input_words.append('</s>')
    target_words = [x for x in target_line.split(' ') if x != '']
    target_words = ['<s>',] + target_words
    target_words.append('</s>')
    
    data.append(input_words)
    data.append(target_words)
    
    #i += 1
    #if i == 10000:
       #break
#print(data)

In [2]:
data[211]

['<s>',
 '贵',
 '人',
 '行',
 '大',
 '礼',
 '，',
 '一',
 '半',
 '福',
 '，',
 '一',
 '半',
 '祥',
 '</s>']

In [3]:
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence
import multiprocessing

cpu_count = multiprocessing.cpu_count() #获取cou数量
vocab_dim = 1024
n_iterations = 200  # ideally more..
n_exposures = 2 # 所有频数超过10的词语
window_size = 7
n_epoch = 4
input_length = 100
maxlen = 100

def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        def parse_dataset(combined): # 闭包-->临时使用
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0) # freqxiao10->0
                data.append(new_txt)
            return data # word=>index
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print('No data provided...')


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined) # input: list
    model.train(combined)
    model.save('./model/Word2vec_model.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined,model

print('Training a Word2vec model...')
index_dict, word_vectors,combined,model=word2vec_train(data)
print('training finish')

unable to import 'smart_open.gcs', disabling that module
C:\Users\16010\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\16010\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
Using TensorFlow backend.
C:\Users\16010\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\16010\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (

Training a Word2vec model...


C:\Users\16010\AppData\Roaming\Python\Python36\site-packages\smart_open\smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


training finish


In [4]:
print(index_dict)

{'晚': 1, '风': 2, '摇': 3, '树': 4, '还': 5, '挺': 6, '</s>': 7, '<s>': 8, '晨': 9, '露': 10, '润': 11, '花': 12, '更': 13, '红': 14, '愿': 15, '景': 16, '天': 17, '成': 18, '无': 19, '墨': 20, '迹': 21, '万': 22, '方': 23, '乐': 24, '奏': 25, '有': 26, '于': 27, '阗': 28, '丹': 29, '枫': 30, '江': 31, '冷': 32, '人': 33, '初': 34, '去': 35, '绿': 36, '柳': 37, '堤': 38, '新': 39, '燕': 40, '复': 41, '来': 42, '忽': 43, '几': 44, '昏': 45, '，': 46, '离': 47, '别': 48, '间': 49, '之': 50, '疾': 51, '病': 52, '不': 53, '及': 54, '终': 55, '年': 56, '同': 57, '静': 58, '好': 59, '茕': 60, '小': 61, '儿': 62, '女': 63, '孱': 64, '羸': 65, '若': 66, '此': 67, '娇': 68, '憨': 69, '烦': 70, '二': 71, '老': 72, '费': 73, '精': 74, '神': 75, '闲': 76, '野': 77, '钓': 78, '稀': 79, '处': 80, '兴': 81, '起': 82, '高': 83, '歌': 84, '酒': 85, '醉': 86, '中': 87, '毋': 88, '负': 89, '我': 90, '下': 91, '虽': 92, '和': 93, '介': 94, '称': 95, '先': 96, '生': 97, '字': 98, '可': 99, '以': 100, '谥': 101, '爱': 102, '臣': 103, '少': 104, '冯': 105, '唐': 106, '为': 107, '郎': 108, '倦': 109, '意': 110, '吾

In [9]:
print(len(word_vectors['你']))
print(word_vectors['你'])
print(word_vectors['我'])

1024
[ 1.3540965   1.1373006  -0.5044759  ... -0.39192748 -0.09534365
 -0.04222712]
[ 0.9339558   0.10886256  0.8365118  ... -1.8102881   0.4075654
 -0.85497016]


In [17]:
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence
import multiprocessing

cpu_count = multiprocessing.cpu_count() #获取cou数量
vocab_dim = 1024
n_iterations = 200  # ideally more..
n_exposures = 2 # 所有频数超过10的词语
window_size = 7
n_epoch = 4
input_length = 100
maxlen = 100

def create_dictionaries(model=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        #def parse_dataset(combined): # 闭包-->临时使用
        #    ''' Words become integers
        #    '''
        #    data=[]
        #    for sentence in combined:
        #        new_txt = []
        #        for word in sentence:
        #            try:
        #                new_txt.append(w2indx[word])
        #            except:
        #                new_txt.append(0) # freqxiao10->0
        #        data.append(new_txt)
        #    return data # word=>index
        #combined=parse_dataset(combined)
        #combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec
    else:
        print('No data provided...')

model = Word2Vec.load('./model/Word2vec_model.pkl')
w2indx,w2vec = create_dictionaries(model)

C:\Users\16010\AppData\Roaming\Python\Python36\site-packages\smart_open\smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
print(w2indx)

{'晚': 1, '风': 2, '摇': 3, '树': 4, '还': 5, '挺': 6, '</s>': 7, '<s>': 8, '晨': 9, '露': 10, '润': 11, '花': 12, '更': 13, '红': 14, '愿': 15, '景': 16, '天': 17, '成': 18, '无': 19, '墨': 20, '迹': 21, '万': 22, '方': 23, '乐': 24, '奏': 25, '有': 26, '于': 27, '阗': 28, '丹': 29, '枫': 30, '江': 31, '冷': 32, '人': 33, '初': 34, '去': 35, '绿': 36, '柳': 37, '堤': 38, '新': 39, '燕': 40, '复': 41, '来': 42, '忽': 43, '几': 44, '昏': 45, '，': 46, '离': 47, '别': 48, '间': 49, '之': 50, '疾': 51, '病': 52, '不': 53, '及': 54, '终': 55, '年': 56, '同': 57, '静': 58, '好': 59, '茕': 60, '小': 61, '儿': 62, '女': 63, '孱': 64, '羸': 65, '若': 66, '此': 67, '娇': 68, '憨': 69, '烦': 70, '二': 71, '老': 72, '费': 73, '精': 74, '神': 75, '闲': 76, '野': 77, '钓': 78, '稀': 79, '处': 80, '兴': 81, '起': 82, '高': 83, '歌': 84, '酒': 85, '醉': 86, '中': 87, '毋': 88, '负': 89, '我': 90, '下': 91, '虽': 92, '和': 93, '介': 94, '称': 95, '先': 96, '生': 97, '字': 98, '可': 99, '以': 100, '谥': 101, '爱': 102, '臣': 103, '少': 104, '冯': 105, '唐': 106, '为': 107, '郎': 108, '倦': 109, '意': 110, '吾